<h1>Chapter12: Pandas advanced apply<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1-分类数据" data-toc-modified-id="1-分类数据-1">1 分类数据</a></span><ul class="toc-item"><li><span><a href="#1.1-背景和目的" data-toc-modified-id="1.1-背景和目的-1.1">1.1 背景和目的</a></span></li><li><span><a href="#1.2-pandas的分类类型" data-toc-modified-id="1.2-pandas的分类类型-1.2">1.2 pandas的分类类型</a></span></li><li><span><a href="#1.3-用分类进行计算" data-toc-modified-id="1.3-用分类进行计算-1.3">1.3 用分类进行计算</a></span></li><li><span><a href="#1.4-用分类提高性能" data-toc-modified-id="1.4-用分类提高性能-1.4">1.4 用分类提高性能</a></span></li><li><span><a href="#1.5-分类方法" data-toc-modified-id="1.5-分类方法-1.5">1.5 分类方法</a></span></li><li><span><a href="#1.6-为建模创建虚拟变量" data-toc-modified-id="1.6-为建模创建虚拟变量-1.6">1.6 为建模创建虚拟变量</a></span></li></ul></li><li><span><a href="#2-GroupBy高级应用" data-toc-modified-id="2-GroupBy高级应用-2">2 GroupBy高级应用</a></span><ul class="toc-item"><li><span><a href="#2.1-分组转换和解封groupby" data-toc-modified-id="2.1-分组转换和解封groupby-2.1">2.1 分组转换和解封groupby</a></span></li><li><span><a href="#2.2-分组的时间重采样" data-toc-modified-id="2.2-分组的时间重采样-2.2">2.2 分组的时间重采样</a></span></li></ul></li><li><span><a href="#3-链式编程技术" data-toc-modified-id="3-链式编程技术-3">3 链式编程技术</a></span><ul class="toc-item"><li><span><a href="#3.1-管道方法" data-toc-modified-id="3.1-管道方法-3.1">3.1 管道方法</a></span></li></ul></li></ul></div>

In [48]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

np.set_printoptions(precision=4, suppress=True)
pd.set_option('display.max_columns', None) # 显示所有列
pd.set_option('display.max_rows', 10) # 设定df全部显示的行数

# 一个cell做多个输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

## 1 分类数据
仔细研究该部分，同时参看有道云笔记中的pandas->series->category datatype部分

### 1.1 背景和目的  
> 分类表示可以在进行分析时大大的提高性能

从数组提取不同的值，并分别计算频率；在数据仓库中，高效的存储和计算，最好的方法是使用所谓的包含不同值的维表(Dimension Table)，将主要的参数存储为引用维表整数键; 分类数据就是根据此思想进行； 

In [49]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)
values

# 唯一值
pd.unique(values)

# 不同值计数
pd.value_counts(values)

values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])
dim.take(values) #  take函数 根据类别内容和顺序构建类别数据 

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

array(['apple', 'orange'], dtype=object)

apple     6
orange    2
dtype: int64

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

### 1.2 pandas的分类类型
- 使用 pd.Series.astype('category') 进行转换
- 使用pd.Categorical(list_type) 进行转换 以上必须给出类别的全部内容
- 使用pd.Categorical.from_codes(codes, categories) 给出类别的单独数据，然后按照codes的顺序指定数据

In [50]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)

df = pd.DataFrame({'fruit': fruits,
            'basket_id': np.arange(N),
            'count': np.random.randint(3, 15,size=N),
            'weight': np.random.uniform(0, 4,size=N)},
            columns=['basket_id', 'fruit', 'count','weight'])

df.head()

fruit_cat = df['fruit'].astype('category') # 将数据转换为category类型
fruit_cat

c = fruit_cat.values
type(c)
c.categories
c.codes

# 将列转换为category
df['fruit'] = df['fruit'].astype('category')
df.fruit

# 或者直接使用pd.Categorical创建  必须给出类别的全部内容
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo','bar']) 
my_categories

,basket_id,fruit,count,weight
0,0,apple,5,3.858058
1,1,orange,8,2.612708
2,2,apple,4,2.995627
3,3,apple,7,2.614279
4,4,apple,12,2.990859


0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

pandas.core.arrays.categorical.Categorical

Index(['apple', 'orange'], dtype='object')

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [51]:
# from_codes 构建类别数据 
categories = ['foo', 'bar', 'baz']  # 默认顺序为012 
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)  # 使用指定的分类编码
my_cats_2

ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True) # 指定有意义的顺序
ordered_cat

# 无序的分类实例可以通过as_ordered进行排序
my_cats_2.as_ordered()


[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

### 1.3 用分类进行计算
对series，可以根据qcut（或者其他方式）生成类别数据，然后根据该类别进行分组组合； 

In [52]:
draws = np.random.randn(1000)
draws[:5]

bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins # 根据qcut生成category类型数据
bins.codes[:10]

bins = pd.Series(bins, name='quartile')
# 根据面元切片结果进行统计  不过可以使用bins新增一列构成成一个df  然后在统计
results = (pd.Series(draws).groupby(bins).agg(['count', 'min', 'max']).reset_index()) 
results

array([ 1.0072, -1.2962,  0.275 ,  0.2289,  1.3529])

[Q4, Q1, Q3, Q3, Q4, ..., Q3, Q3, Q2, Q4, Q3]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

array([3, 0, 2, 2, 3, 3, 0, 1, 3, 1], dtype=int8)

,quartile,count,min,max
0,Q1,250,-2.949343,-0.697614
1,Q2,250,-0.694838,-0.017007
2,Q3,250,-0.011862,0.625428
3,Q4,250,0.627076,3.927528


### 1.4 用分类提高性能 
将数据转换为分类数据可以极大的提高效率，但其类别的设定需要满足需求，或者直接生成类别数

In [53]:
# 千万级数据使用类别标签
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))  # 类别的设定 只是现实数据中没有这么简单

# 将标签转换为分类数据
categories = labels.astype('category')

# 使用内存显示，类别数据极大降低了使用内存
labels.memory_usage()

categories.memory_usage()


80000080

10000272

In [54]:
# 转换为分类不是没有代价的，但这是一次性的代价
%time _ = labels.astype('category')

Wall time: 671 ms


### 1.5 分类方法 
- 使用类别的cat属性提供方便的分类和编码的使用方法
- 使用set_categories方法改变分类属性
- 可以使用remove_unused_categories方法删除没看到的分类   

**常用的分类方法：**

方法|说明
--|--
add_categories|在已存在的分类后面添加新的（未使用的）分类
as_ordered|使分类有序
as_unordered|使分类无序
remove_categories|移除分类，设置任何被移除的值为null
remove_unused_categories|移除任意不出现在数据中的分类值
rename_categories|用指定的新分类的名字替换分类，不能改变分类的数目
reorder_catagories|与rename_categories类似，但是可以改变结果，使分类有序
set_categpries|用指定的新分类的名字替换分类，可以添加或者删除分类

In [55]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

# 使用cat属性查看数据和类别
cat_s.cat.codes
cat_s.cat.categories

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

Index(['a', 'b', 'c', 'd'], dtype='object')

In [56]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)  # 改变分类属性
cat_s2

cat_s.value_counts()
cat_s2.value_counts()

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

d    2
c    2
b    2
a    2
dtype: int64

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [57]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

### 1.6 为建模创建虚拟变量
主要为one-hot变量转换，书中介绍的比较简单，具体的参考有道云笔记的特征工程文件目录

In [58]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

# get_dummies 函数构建series的分布矩阵
pd.get_dummies(cat_s) # 可以转换这个分类数据为包含虚拟变量的DataFrame

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## 2 GroupBy高级应用  
该部分很重要，需要补充其他文档，具体参看有道云比较的pandas groupby系列
### 2.1 分组转换和解封groupby 

transform方法与apply的差别：
- 它可以产生向分组形状广播标量值
- 它可以产生一个和输入组形状相同的对象
- 它不能修改输人

```python
help(pd.DataFrame.transform)
```

In [59]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,'value': np.arange(12.)})
df.head()

g = df.groupby('key').value
g.mean()
g.transform(lambda x: x.mean())

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0


key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
     ... 
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, Length: 12, dtype: float64

In [60]:
g.transform(lambda x: x.rank(ascending=False))

def normalize(x):
    return (x - x.mean()) / x.std()
g.transform(normalize)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
     ... 
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, Length: 12, dtype: float64

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
        ...   
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, Length: 12, dtype: float64

### 2.2 分组的时间重采样
resample方法从语义上是一个基于内在时间的分组操作;也可以使用pd.TimeGrouper构建重采样对象，然后依次根据此进行分组

In [61]:
N = 15
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)

df = pd.DataFrame({'time': times, 'value': np.arange(N)})
df.head()

# 可以用time作为索引，然后重采样
df.set_index('time').resample('5min').count()

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4


,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [62]:
df2 = pd.DataFrame({'time': times.repeat(3),
            'key': np.tile(['a', 'b', 'c'], N),
            'value': np.arange(N * 3.)})

df2.head()

time_key = pd.Grouper(freq='5min') # 使用TimeGrouper构建重采样对象 不过已经被pd.Grouper方法代替
resampled = (df2.set_index('time').groupby(['key', time_key]).sum())
resampled.head()
resampled.reset_index().head()

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0


value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0


## 3 链式编程技术
所谓链式编程，就是尽量不使用中间临时变量，多个函数组合在一起使用；
```python
result = df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())
         .groupby('key')
         .col1_demeaned.std()
```
这种写法可以省略中间变量以节省部分的内存空间，但是程序可读性会受到影响，建议计算及转换部分使用该方法，其他还是使用中间临时变量保存数据；

### 3.1 管道方法
> f(df)和df.pipe(f)是等价的，但是pipe使得链式声明更容易;   

主要使用pipe方法方便构建链式编程
```python
result = (df.pipe(f, arg1=v1)
         .pipe(g, v2, arg3=v3)
         .pipe(h, arg4=v4))
```

In [63]:
## 使用pipe对复杂转换进行链式编程
def group_demean(df, by, cols):
    result = df.copy()
    g = df.groupby(by)
    for c in cols:
        result[c] = df[c] - g[c].transform('mean')
    return result

# result = (df[df.col1 < 0].pipe(group_demean, ['key1', 'key2'], ['col1']))